# Import the needed libraries

In [ ]:
# Import the packages we will need
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3

# Load the Wormbase GeneIDs
This file is exported from wormbase.org

In [ ]:
gene_ids_df = pd.read_csv('./data/WS289.geneIDs.csv') 
gene_ids_df.set_index('Wormbase_Id', inplace=True)
gene_ids_df.columns

# Load the Excel file to work with
* This file should have alteast one "Tab"
* And the first column of the sheet should have Wormbase IDs

========
* The column layout is as follows: 
* `ID,	Sequence_Name,	Gene_Name`

# Load the Excel with the Gene Sets of interest


In [ ]:
# Read in the Candidate_Genes extracted from from Brendans Spreadsheet

xlsx_file_nm = './data/KB_stress gene expression.xlsx'
genes_xlsx = pd.ExcelFile(xlsx_file_nm)
sheet_names = genes_xlsx.sheet_names
sheet_names

In [ ]:
# Add the dataframes to a dictionary keyed by the sheet name
genes_dfs = {}
for sheet_name in sheet_names:
    sheet_df = pd.read_excel(xlsx_file_nm, sheet_name=sheet_name)
    genes_dfs[sheet_name] = sheet_df
#genes_dfs
genes_dfs.keys()

In [ ]:
# Replace the Gene_Names with those from Wormbase to ensure they are correct

for sheet_name in genes_dfs.keys():
    sheet_df = genes_dfs[sheet_name]
    # Drop the empty Gene_Name column
    sheet_df.drop(columns='Sequence_Name', inplace=True)
    sheet_df.drop(columns='Gene_Name', inplace=True)
    # Add Gene_Name from the Wormbase geneIDs list
    sheet_df.set_index('ID', inplace=True)
    sheet_df = pd.merge(sheet_df, gene_ids_df[['Sequence_id']], left_index=True, right_index=True)
    sheet_df = pd.merge(sheet_df, gene_ids_df[['Gene_name']], left_index=True, right_index=True)
    genes_dfs[sheet_name] = sheet_df 
#genes_dfs

# Load the UP and Down Genes to Compare against
* The should be the output from DE Seq 

In [ ]:
up_genes_df = pd.read_csv('./data/N2_PQ_DOWN.csv') 
down_genes_df = pd.read_csv('./data/N2_PQ_UP.csv') 
up_down_genes_df = pd.concat([up_genes_df, down_genes_df])

print(len(up_down_genes_df))


In [ ]:
# Function to create and save the Venn Diagram

def venn_plot(set1_df, set2_df, set3_dfs, names, filename):
    print(names)
    print(names[2])
    set1 = set1_df['ID']
    print(len(set1))
    set2 = set2_df['ID']
    print(len(set2))
    print(set3_dfs.keys())
    print(names[2] in set3_dfs.keys())
    set3_df = set3_dfs[str(names[2])]
    print(set3_df)
    set3 = set3_df.index
    print(len(set3))
    
    
    # Convert Series to sets
    set1 = set(set1)
    set2 = set(set2)
    set3 = set(set3)

    # Create the Venn diagram
    venn = venn3([set1, set2, set3], names)

    # Add commas to numbers and set them as labels for the circles
    #venn.get_label_by_id('100').set_text(f"{len(set1):,}")
    #venn.get_label_by_id('010').set_text(f"{len(set2):,}")
    #venn.get_label_by_id('001').set_text(f"{len(set3):,}")

    # Save the plot
    plt.title(f"Venn Diagram {names[0]}, {names[1]} and {names[2]}")
    plt.savefig(filename)
    plt.close()
    return


In [ ]:
# Function to find the overlaping genes and create a column in the dataframe representing the overlap

def find_overlap(set1_df, set2_df, set3_dfs, names, filename):
    set1 = set1_df['ID']
    set2 = set2_df['ID']
    set3 = set3_dfs[names[2]].index

    # Convert Pandas Series to sets
    set1 = set(set1)
    set2 = set(set2)
    set3 = set(set3)

    # Find elements that overlap between Set1 and Set3
    overlap_set1_set3 = set1.intersection(set3)

    # Find elements that overlap between Set2 and Set3
    overlap_set2_set3 = set2.intersection(set3)

    # Find elements that overlap between Set1, Set2 and Set3
    overlap_all_sets = set1.intersection(set2, set3)

    set3_dfs[names[2]][names[0]] = set3_dfs[names[2]].index.isin(overlap_set1_set3)
    set3_dfs[names[2]][names[1]] = set3_dfs[names[2]].index.isin(overlap_set2_set3)
    set3_dfs[names[2]]['Both'] = set3_dfs[names[2]].index.isin(overlap_all_sets)
    
    set3_dfs[names[2]] = set3_dfs[names[2]].sort_values(by=['Both', names[1], names[0]], ascending=[False, False, False])

    
    return set3_dfs[names[2]]

In [ ]:
# Highlight the Cells of interest and Autofit the columns

from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

def highlight_rows_by_column(worksheet, column_number, color):
    color_fill = PatternFill(start_color=color, end_color=color, fill_type='solid')
    df = pd.DataFrame(worksheet.values)
    for row_idx, row in enumerate(df.itertuples(), start=1):
        if isinstance(row[column_number], bool) and row[column_number] is True:  # Check if the column value is True
            for col_idx in range(1, len(row)):
                worksheet.cell(row=row_idx, column=col_idx).fill = color_fill

def autofit_columns(worksheet):
    for column in worksheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2  # Adding some extra padding
        worksheet.column_dimensions[get_column_letter(column[0].column)].width = adjusted_width


In [ ]:
# Put it all together and generate the output
from openpyxl import Workbook

excel_file_path = "./data/Genes_Overlap1.xlsx"
if os.path.exists(excel_file_path):
    os.remove(excel_file_path)

# Create a new workbook
workbook = Workbook()

# Remove the default "Sheet" created by openpyxl
#default_sheet = workbook['Sheet']
#workbook.remove(default_sheet)

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    #writer.book = Workbook()
    for sheet_name in sheet_names:
        names = ('N2_PQ_Up', 'N2_PQ_Down', sheet_name)
        filename = f"./data/venn_{sheet_name}.png"    
        plot = venn_plot(up_genes_df, down_genes_df, genes_dfs, names, filename)
        sheet_df = find_overlap(up_genes_df, down_genes_df, genes_dfs, names, filename)
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=True, index_label='ID')
        
    # Autofit and highlight columns for each sheet
    for sheet in writer.sheets.values():
        autofit_columns(sheet)
        for index, color in  enumerate(['fde9d9','daeef3','e4dfec']):
            highlight_rows_by_column(sheet, index+4, color)
            


# Appendix   

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows  # Importing dataframe_to_rows

excel_file_path = './data/output.xlsx'

# Create a DataFrame (example data)
data = {'Column1': [1, 2, 3], 'Column2': ['A', 'B', 'C']}
df = pd.DataFrame(data)

# Create a Workbook instance using openpyxl
workbook = Workbook()

# Access the active sheet (or create a new sheet) in the workbook
sheet = workbook.active

# Write the DataFrame to the sheet in the workbook
for r in dataframe_to_rows(df, index=False, header=True):
    sheet.append(r)

# Save the workbook to the desired file path
workbook.save(excel_file_path)



### Convert Gene Names to Wormbase IDs

In [ ]:
gene_ids_df = pd.read_csv('./data/WS289.geneIDs.csv') 
gene_names_df = pd.read_csv('./data/gene_names.csv')

print(gene_ids_df.columns[1])
print(gene_names_df.columns[0])
print(gene_ids_df.columns[1]==gene_names_df.columns[0])
print(len(gene_names_df))

In [ ]:
# Match Sequence Ids that are in the Gene_name Column
merged_df = pd.merge(gene_names_df, gene_ids_df, left_on='Gene_name', right_on='Sequence_id', how='left')
columns_to_drop = ['Gene_name_y', 'Sequence_id','Gene_Type']
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df = merged_df.rename(columns={'Gene_name_x': 'Gene_name'})
merged_df

In [ ]:
# Match the Gene_names that are in the Gene_name Column
merged_df1 = pd.merge(merged_df, gene_ids_df, left_on='Gene_name', right_on='Gene_name', how='left')
merged_df1

In [ ]:
# Combimed the Wormbase_Id Ids the Matched above into a Single Column 
merged_df1['Wormbase_Id'] = merged_df1['Wormbase_Id_x']
merged_df1.loc[merged_df['Wormbase_Id'].isnull(), 'Wormbase_Id'] = merged_df1['Wormbase_Id_y']
merged_df1

In [ ]:
# Drop an rows that still have null values and save
has_values = merged_df1[~merged_df1['Wormbase_Id'].isnull()]
has_values.to_csv('./data/has_values.csv')
has_values

In [ ]:
# Print the remaining rows that do not have Wormbase IDs
nan_values = merged_df1[merged_df1['Wormbase_Id'].isnull()]
print(len(nan_values))
nan_values['Gene_name']
